## References and important URLS requirements- 

- [1] https://openai.com/blog/function-calling-and-other-api-updates
- [2] https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb
- [3] https://rapidapi.com/weatherapi/api/weatherapi-com/
- [4] https://platform.openai.com/account/api-keys

In [1]:
import os
ROOT = "/Users/harik/OneDrive/Desktop/ai/gpt_function_calling/"
os.getcwd()

'c:\\Users\\harik\\OneDrive\\Desktop\\ai\\gpt_function_calling'

# Previous ways - 

## Via Openai Python ChatCompletion class- 

```python
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)
```

## Via CURL command

```bash
curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-3.5-turbo",
    "messages": [
      {"role": "system", "content": "You are a helpful assistant."}, 
      {"role": "user", "content": "Hello!"}
      ]
  }'
```

## Via requests module

```python
import requests

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer " + OPENAI_API_KEY,
}
json_data = {
    "model": "gpt-3.5-turbo", 
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": "Hello!"}
        ]}
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=json_data,
)
```


Weather

## Weather Chatbot Example


In [2]:
import requests
from dotenv import load_dotenv
import json
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GPT_MODEL ="gpt-3.5-turbo-0613" # new model for function calling capablities


In [3]:
import requests

def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):

    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + OPENAI_API_KEY,
    }
    json_data = {
        "model": model, 
        "messages": messages, 
        }
    
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
        
    try:    
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        
        return response
    except Exception as e:
        print(f"\n>>> Unable to generate Chat Completion Response")
        print(f"/n>>> Exception: {e}")
        raise e
        

In [4]:
# sample use

sample_messages = [
    {"role":"system", "content":"you are a helpful assistant"},
    {"role":"user", "content":"Hi there"}
]
chat_response = chat_completion_request(messages=sample_messages)

chat_response

<Response [200]>

In [5]:
chat_response.json()

{'id': 'chatcmpl-8OqIeSWkrNeqAwQRVkllh4H9FxWeF',
 'object': 'chat.completion',
 'created': 1700931604,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello! How can I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 18, 'completion_tokens': 9, 'total_tokens': 27}}

In [6]:
assistant_message = chat_response.json()["choices"][0]["message"]
assistant_message 

{'role': 'assistant', 'content': 'Hello! How can I assist you today?'}

In [7]:
creator_name = "demohari"

system_instruction = f"""
You are Weather bot created by {creator_name}, \
You first greet the customer, and you MUST introduce yourself as -
"Hi I am Weather bot created by {creator_name}! How may I assist you today?" \
an automated service to tell Weather infomation \
based on the location provided by the user. \
then collects the location information\
Don't make assumptions about what \
values to plug into functions. \
Ask for clarification if a user request is ambiguous.
You respond in a short, very conversational friendly style. \
"""

In [8]:
messages =[
    {"role":"system", "content":system_instruction}
]

In [9]:
functions = [
    {
        "name": "get_current_weather", # name of your function
        "description": "Get the current weather", # description of your function
        "parameters": {
            "type": "object",
            "properties": { # 1st argument of your function
                "location": { # argument name
                    "type": "string", # argument type
                    "description": "The city and state, e.g. San Francisco, CA", # description of your function with example
                }
            },
            "required": ["location"], # required argument of your
        },
    }
]

In [25]:
def get_current_weather(location):
    RAPID_API_KEY = os.getenv("RAPID_API_KEY")
    try:
        url = "https://weatherapi-com.p.rapidapi.com/current.json"

        querystring = {"q": location}
        print(f"\n>>>> got the querystring as: {querystring}")
        headers = {
            "X-RapidAPI-Key": RAPID_API_KEY,
            "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        response_json = response.json()
        print(f"\n>>>> got the RAPID API response as: \n{response_json}")
        return response_json
    except Exception as e:
        raise e

In [26]:
user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)

In [27]:
chat_response

<Response [200]>

In [28]:
chat_response.json()

{'id': 'chatcmpl-8OqOldgy7IkQJ0ZG09HmeHCSKCZYp',
 'object': 'chat.completion',
 'created': 1700931983,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hi! I am Weather bot created by demohari. How may I assist you today?'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 690, 'completion_tokens': 20, 'total_tokens': 710}}

In [29]:
assistant_message=chat_response.json()["choices"][0]["message"]
assistant_message

{'role': 'assistant',
 'content': 'Hi! I am Weather bot created by demohari. How may I assist you today?'}

In [30]:
messages = list()
messages.append({"role": "system", "content": system_instruction})


user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)
print(f"\n>>>> complete_chat_response: \n{chat_response.json()}")
assistant_message = chat_response.json()["choices"][0]["message"]
print(f"\n>>>> assistant message: \n{assistant_message}")


>>>> complete_chat_response: 
{'id': 'chatcmpl-8OqOy8ryXANlTpRPLosaIph9a3B6u', 'object': 'chat.completion', 'created': 1700931996, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Hi! I am Weather bot created by demohari! How may I assist you today?'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 163, 'completion_tokens': 20, 'total_tokens': 183}}

>>>> assistant message: 
{'role': 'assistant', 'content': 'Hi! I am Weather bot created by demohari! How may I assist you today?'}


In [31]:
messages.append(assistant_message) # to give context
messages

[{'role': 'system',
  'content': '\nYou are Weather bot created by demohari, You first greet the customer, and you MUST introduce yourself as -\n"Hi I am Weather bot created by demohari! How may I assist you today?" an automated service to tell Weather infomation based on the location provided by the user. then collects the location informationDon\'t make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.\nYou respond in a short, very conversational friendly style. '},
 {'role': 'user', 'content': 'Hi there!'},
 {'role': 'assistant',
  'content': 'Hi! I am Weather bot created by demohari! How may I assist you today?'}]

In [46]:
user_message = "I want to know the weather at hyderabad IN"
messages.append({"role": "user", "content": user_message})
chat_response = chat_completion_request(messages = messages, functions = functions)
print(f"\n>>>> complete_chat_response: \n{chat_response.json()}\n")
assistant_message = chat_response.json()["choices"][0]["message"]
print(f"\n>>>> assistant message: \n{assistant_message}")


>>>> complete_chat_response: 
{'id': 'chatcmpl-8OqSFJ8eWKommjgIo4laYWmkB4q4T', 'object': 'chat.completion', 'created': 1700932199, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Hyderabad, IN"\n}'}}, 'finish_reason': 'function_call'}], 'usage': {'prompt_tokens': 1137, 'completion_tokens': 19, 'total_tokens': 1156}}


>>>> assistant message: 
{'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Hyderabad, IN"\n}'}}


In [47]:
assistant_message.get("function_call")

{'name': 'get_current_weather',
 'arguments': '{\n  "location": "Hyderabad, IN"\n}'}

In [48]:
assistant_message.get("function_call").get("name") # string

'get_current_weather'

In [49]:
assistant_message.get("function_call").get("arguments") # string

'{\n  "location": "Hyderabad, IN"\n}'

In [50]:
import json

def execute_function_call(assistant_message):
    if assistant_message.get("function_call").get("name") == "get_current_weather":
        location = json.loads(assistant_message.get("function_call").get("arguments") )["location"]
        results = get_current_weather(location)
    else:
        results = f"Error: function {assistant_message['function_call']['name']} does not exist"

    return results

In [51]:
results = execute_function_call(assistant_message)
content = json.dumps(results)
content


>>>> got the querystring as: {'q': 'Hyderabad, IN'}

>>>> got the RAPID API response as: 
{'location': {'name': 'Hyderabad', 'region': 'Andhra Pradesh', 'country': 'India', 'lat': 17.38, 'lon': 78.47, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1700932160, 'localtime': '2023-11-25 22:39'}, 'current': {'last_updated_epoch': 1700931600, 'last_updated': '2023-11-25 22:30', 'temp_c': 22.0, 'temp_f': 71.6, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 117, 'wind_dir': 'ESE', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.02, 'precip_in': 0.0, 'humidity': 83, 'cloud': 75, 'feelslike_c': 24.6, 'feelslike_f': 76.2, 'vis_km': 5.0, 'vis_miles': 3.0, 'uv': 1.0, 'gust_mph': 9.5, 'gust_kph': 15.3}}


'{"location": {"name": "Hyderabad", "region": "Andhra Pradesh", "country": "India", "lat": 17.38, "lon": 78.47, "tz_id": "Asia/Kolkata", "localtime_epoch": 1700932160, "localtime": "2023-11-25 22:39"}, "current": {"last_updated_epoch": 1700931600, "last_updated": "2023-11-25 22:30", "temp_c": 22.0, "temp_f": 71.6, "is_day": 0, "condition": {"text": "Mist", "icon": "//cdn.weatherapi.com/weather/64x64/night/143.png", "code": 1030}, "wind_mph": 2.2, "wind_kph": 3.6, "wind_degree": 117, "wind_dir": "ESE", "pressure_mb": 1019.0, "pressure_in": 30.09, "precip_mm": 0.02, "precip_in": 0.0, "humidity": 83, "cloud": 75, "feelslike_c": 24.6, "feelslike_f": 76.2, "vis_km": 5.0, "vis_miles": 3.0, "uv": 1.0, "gust_mph": 9.5, "gust_kph": 15.3}}'

In [52]:
def get_natural_response(content):
    convert_prompt = f"convert this results from weather api to a natural english sentence: {content}"
    messages.append({"role": "user", "content": convert_prompt})
    convert_prompt_response = chat_completion_request(messages=messages)
    print(f"\n>>>> recieved message: {convert_prompt_response.json()}")
    new_assistant_message = convert_prompt_response.json()["choices"][0]["message"]
    messages.append(new_assistant_message)
    content = new_assistant_message["content"]
    print(f"\n>>>> natural response: \n{content}")
    return content


content = get_natural_response(content)


>>>> recieved message: {'id': 'chatcmpl-8OqSVEGaI3R3p6JWmNzYv9GZkqumd', 'object': 'chat.completion', 'created': 1700932215, 'model': 'gpt-3.5-turbo-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "The current weather in Hyderabad, India is misty. It's currently 22 degrees Celsius (71.6 degrees Fahrenheit) with a wind speed of 3.6 kilometers per hour coming from the east-southeast direction. The pressure is 1019.0 millibars, and the humidity level is 83%. The sky is 75% covered with clouds. Visibility is slightly reduced to 5.0 kilometers (3.0 miles) due to mist. It feels like 24.6 degrees Celsius (76.2 degrees Fahrenheit)."}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1429, 'completion_tokens': 112, 'total_tokens': 1541}}

>>>> natural response: 
The current weather in Hyderabad, India is misty. It's currently 22 degrees Celsius (71.6 degrees Fahrenheit) with a wind speed of 3.6 kilometers per hour coming from the east-southeast direction. The pr